## **Text summarization using T5**




---



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow
tensorflow.__version__

'2.4.1'

In [ ]:
!pip install --upgrade tensorflow

Requirement already up-to-date: tensorflow in /usr/local/lib/python3.7/dist-packages (2.4.1)


In [ ]:
!pip install transformers==2.9.1

     |████████████████████████████████| 645kB 5.5MB/s 
     |████████████████████████████████| 890kB 6.9MB/s 
     |████████████████████████████████| 1.2MB 15.9MB/s 
     |████████████████████████████████| 5.6MB 13.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=32e9350a6c8eb9085021e80671a01a40e16e01571b848ef9f4700b1e405a4297
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
!pip install sentencepiece

In [ ]:

import tensorflow as tf
from transformers import T5Tokenizer, TFT5Model, TFT5ForConditionalGeneration
import tensorflow_datasets as tfds
import time

In [ ]:
import os

BASE = os.path.join('drive', 'MyDrive', 'BERT_abstractive_summarization')
log_file = os.path.join(BASE, 'tensorflow.log')
saved_weight = os.path.join(BASE, 'saved_weight_model_large')

In [ ]:

BATCH_SIZE = 8

SHUFFEL_SIZE = 1024

learning_rate = 3e-5

In [ ]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = TFT5ForConditionalGeneration.from_pretrained('t5-base')
model.load_weights(saved_weight)
task_specific_params = {
    "summarization": {
      "early_stopping": True,
      "length_penalty": 2.0,
      "max_length": 500,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    }
}
if task_specific_params is not None:
    model.config.update(task_specific_params.get("summarization", {}))
    
pad_token_id = tokenizer.pad_token_id

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08, clipnorm=1.0)
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

val_loss = tf.keras.metrics.Mean(name='val_loss')
val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='val_accuracy')

model.summary()

Model: "tf_t5for_conditional_generation"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
shared (TFSharedEmbeddings)  multiple                  24674304  
_________________________________________________________________
encoder (TFT5MainLayer)      multiple                  84954240  
_________________________________________________________________
decoder (TFT5MainLayer)      multiple                  113275392 
Total params: 222,903,936
Trainable params: 222,903,936
Non-trainable params: 0
_________________________________________________________________


In [ ]:

cnn_dailymail = tfds.load(name="cnn_dailymail")

INFO:absl:No config specified, defaulting to first: cnn_dailymail/plain_text
INFO:absl:Load pre-computed DatasetInfo (eg: splits, num examples,...) from GCS: cnn_dailymail/plain_text/3.0.0
INFO:absl:Load dataset info from /tmp/tmptsn0hq69tfds
INFO:absl:Field info.description from disk and from code do not match. Keeping the one from code.
INFO:absl:Field info.config_name from disk and from code do not match. Keeping the one from code.
INFO:absl:Field info.config_description from disk and from code do not match. Keeping the one from code.
INFO:absl:Field info.citation from disk and from code do not match. Keeping the one from code.
INFO:absl:Generating dataset cnn_dailymail (/root/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0)


INFO:absl:Downloading https://drive.google.com/uc?export=download&id=0BwmD_VLjROrfTHk4NFg2SndKcjQ into /root/tensorflow_datasets/downloads/ucexport_download_id_0BwmD_VLjROrfTHk4NFg2SndKG8BdJPpt2iRo6Dpzz23CByJuAePEilB-pxbcBCHaWDs.tmp.b492177db7d945afbb6d6817e2a6c065...
INFO:absl:Downloading https://drive.google.com/uc?export=download&id=0BwmD_VLjROrfM1BxdkxVaTY2bWs into /root/tensorflow_datasets/downloads/ucexport_download_id_0BwmD_VLjROrfM1BxdkxVaTY2zVV-G71RIXPssrrvSAjt19Cy91r-9CQ2F9DMKA0uFk0.tmp.f9299e2d8ac64528838a19c954c28046...
INFO:absl:Downloading https://raw.githubusercontent.com/abisee/cnn-dailymail/master/url_lists/all_test.txt into /root/tensorflow_datasets/downloads/raw.gith.com_abis_cnn-dail_mast_url_list_a705_isK790OHOPsIZX-ACsObq_vchU9r5Uduh6ULX3c.txt.tmp.14bf4d9036a644dcbcf9305110218f77...
INFO:absl:Downloading https://raw.githubusercontent.com/abisee/cnn-dailymail/master/url_lists/all_train.txt into /root/tensorflow_datasets/downloads/raw.gith.com_abis_cnn-dail_mast_url

INFO:absl:Generating split train


Shuffling and writing examples to /root/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0.incompleteEY32GK/cnn_dailymail-train.tfrecord


INFO:absl:Done writing /root/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0.incompleteEY32GK/cnn_dailymail-train.tfrecord. Shard lengths: [17945, 17944, 17945, 17944, 17945, 17944, 17945, 17944, 17945, 17945, 17944, 17945, 17944, 17945, 17944, 17945]
INFO:absl:Generating split validation


Shuffling and writing examples to /root/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0.incompleteEY32GK/cnn_dailymail-validation.tfrecord


INFO:absl:Done writing /root/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0.incompleteEY32GK/cnn_dailymail-validation.tfrecord. Shard lengths: [13368]
INFO:absl:Generating split test


Shuffling and writing examples to /root/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0.incompleteEY32GK/cnn_dailymail-test.tfrecord


INFO:absl:Done writing /root/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0.incompleteEY32GK/cnn_dailymail-test.tfrecord. Shard lengths: [11490]
INFO:absl:Skipping computing stats for mode ComputeStatsMode.SKIP.
INFO:absl:Constructing tf.data.Dataset for split None, from /root/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0


Dataset cnn_dailymail downloaded and prepared to /root/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0. Subsequent calls will reuse this data.


In [ ]:

train_tfds = cnn_dailymail['train']
test_tfds = cnn_dailymail['test']
val_tfds = cnn_dailymail['validation']


In [ ]:
len_train = len(list(train_tfds))
len_test = len(list(test_tfds))
len_val = len(list(val_tfds))
print(f'Train samples: {len_train} - Test samples: {len_test} - Validation samples: {len_val}')

Train samples: 287113 - Test samples: 11490 - Validation samples: 13368


In [ ]:
def normalize_text(text):
    """Lowercase and remove quotes from a TensorFlow string."""
    # text = tf.strings.lower(text)
    # text = tf.strings.regex_replace(text,"'(.*)'", r"\1")
    return text.numpy().decode('UTF-8')

def tokenize_articles(text):
    text = normalize_text(text)
    ids = tokenizer.encode_plus((model.config.prefix + text), return_tensors="tf", max_length= 512) 
    
    return tf.squeeze(ids['input_ids']), tf.squeeze(ids['attention_mask'])
        
def tokenize_highlights(text):
    text = normalize_text(text)
    ids = tokenizer.encode(text, return_tensors="tf", max_length= 150)
    return tf.squeeze(ids)



def map_func(features):

    article_ids, attention_mask = tf.py_function(tokenize_articles, inp=[features["article"]], Tout=(tf.int32, tf.int32))
    highlights_ids = tf.py_function(tokenize_highlights, inp=[features["highlights"]], Tout=tf.int32)

    return article_ids, attention_mask, highlights_ids



In [ ]:
train_ds = train_tfds.map(map_func)\
    .shuffle(SHUFFEL_SIZE)\
    .padded_batch(BATCH_SIZE, padded_shapes=([512],[512],[150]))\
    .prefetch(tf.data.experimental.AUTOTUNE)

val_ds = val_tfds.map(map_func)\
    .shuffle(SHUFFEL_SIZE)\
    .padded_batch(BATCH_SIZE, padded_shapes=([512],[512],[150]))\
    .prefetch(tf.data.experimental.AUTOTUNE)

test_ds = test_tfds.map(map_func)\
    .shuffle(SHUFFEL_SIZE)\
    .padded_batch(BATCH_SIZE, padded_shapes=([512],[512],[150]))\
    .prefetch(tf.data.experimental.AUTOTUNE)



In [ ]:
@tf.function
def train_step(input_ids, input_mask, y):
    # https://github.com/huggingface/transformers/blob/master/examples/summarization/bart/finetune.py
    y_ids = y[:, :-1]
    lm_labels = tf.identity(y[:, 1:])
    lm_labels = tf.where(tf.equal(y[:, 1:],pad_token_id), -100, lm_labels)

    with tf.GradientTape() as tape:
        # prediction_scores: (bs, 150, 32128)
        # decoder_past_key_value_states: (bs, 512, 512), (bs, 8, 150, 64)
        # z: (bs, 512, 512)
        predictions, _, _ = model(input_ids, attention_mask=input_mask, decoder_input_ids=y_ids, lm_labels=lm_labels, training=True)
        loss = loss_object(y[:, 1:], predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(y[:, 1:], predictions)

In [ ]:

@tf.function
def val_step(input_ids, input_mask, y):
    # https://github.com/huggingface/transformers/blob/master/examples/summarization/bart/finetune.py
    y_ids = y[:, :-1]
    lm_labels = tf.identity(y[:, 1:])
    lm_labels = tf.where(tf.equal(y[:, 1:],pad_token_id), -100, lm_labels)
    # with tf.device('/cpu:0'):
    predictions, _, _ = model(input_ids, attention_mask=input_mask, decoder_input_ids=y_ids, lm_labels=lm_labels, training=False)
    v_loss = loss_object(y[:, 1:], predictions)

    val_loss(v_loss)
    val_accuracy(y[:, 1:], predictions)

In [ ]:
import logging

log = logging.getLogger('tensorflow')
log.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fh = logging.FileHandler(log_file)
fh.setLevel(logging.DEBUG)
fh.setFormatter(formatter)
log.addHandler(fh)
log.propagate = True

In [ ]:
EPOCHS = 2
log_interval = 200
for epoch in range(EPOCHS):
    log.info(f'START {epoch}')
    # reset metrics
    train_loss.reset_states()
    train_accuracy.reset_states()
    
    val_loss.reset_states()
    val_accuracy.reset_states()
    
    val_batches = iter(val_ds)
    
    start_time = time.time()
    for i, (input_ids, input_mask, y) in enumerate(train_ds):
        # training
        train_step(input_ids, input_mask, y)
        
        # validation
        if i % log_interval == 0:
            x_val, x_mask_val, y_val = next(val_batches)
            # with tf.device('/cpu:0'):
            val_step(x_val, x_mask_val, y_val)
            elapsed = time.time() - start_time
            log.info('| epoch {:3d} | [{:5d}/{:5d}] | '
                  'ms/batch {:5.2f} | '
                  'train acc {:5.2f} | val acc {:5.2f} |'
                  'loss {:5.2f} | val loss {:5.2f}'.format(
                    epoch, i, int(len_train/8),
                    elapsed * 1000 / log_interval,
                    train_accuracy.result() * 100, val_accuracy.result() * 100, 
                    train_loss.result(),  val_loss.result()))
            model.save_weights(saved_weight)
            log.info(f'SAVED WEIGHTS TO {saved_weight}')
            start_time = time.time()
    log.info(f'END {epoch}')
    

INFO:tensorflow:START 0


INFO:tensorflow:START 0


INFO:tensorflow:| epoch   0 | [    0/35889] | ms/batch 189.53 | train acc 79.53 | val acc 82.05 |loss  0.94 | val loss  0.82


INFO:tensorflow:| epoch   0 | [    0/35889] | ms/batch 189.53 | train acc 79.53 | val acc 82.05 |loss  0.94 | val loss  0.82


INFO:tensorflow:SAVED WEIGHTS TO drive/MyDrive/BERT_abstractive_summarization/saved_weight_model_large


...

INFO:tensorflow:| epoch   0 | [10800/35889] | ms/batch 825.77 | train acc 84.06 | val acc 81.73 |loss  0.72 | val loss  0.86


INFO:tensorflow:| epoch   0 | [10800/35889] | ms/batch 825.77 | train acc 84.06 | val acc 81.73 |loss  0.72 | val loss  0.86


INFO:tensorflow:SAVED WEIGHTS TO drive/MyDrive/BERT_abstractive_summarization/saved_weight_model_large


INFO:tensorflow:SAVED WEIGHTS TO drive/MyDrive/BERT_abstractive_summarization/saved_weight_model_large


KeyboardInterrupt: ignored

In [ ]:
!pip install rouge-score

In [ ]:
from rouge_score import rouge_scorer
from rouge_score import scoring

class RougeScore:
    '''
    mostly from https://github.com/google-research/text-to-text-transfer-transformer/blob/master/t5/evaluation/metrics.py 
    '''
    
    def __init__(self, score_keys=None)-> None:
        super().__init__()
        if score_keys is None:  
            self.score_keys = ["rouge1", "rouge2", "rougeLsum"]
        
        self.scorer = rouge_scorer.RougeScorer(self.score_keys)
        self.aggregator = scoring.BootstrapAggregator()
        
        
    @staticmethod
    def prepare_summary(summary):
            # Make sure the summary is not bytes-type
            # Add newlines between sentences so that rougeLsum is computed correctly.
            summary = summary.replace(" . ", " .\n")
            return summary
    
    def __call__(self, target, prediction):
        """Computes rouge score.''
        Args:
        targets: string
        predictions: string
        """

        target = self.prepare_summary(target)
        prediction = self.prepare_summary(prediction)
        
        self.aggregator.add_scores(self.scorer.score(target=target, prediction=prediction))

        return 
    
    def reset_states(self):
        self.rouge_list = []

    def result(self):
        result = self.aggregator.aggregate()
        
        for key in self.score_keys:
            score_text = "%s = %.2f, 95%% confidence [%.2f, %.2f]"%(
                key,
                result[key].mid.fmeasure*100,
                result[key].low.fmeasure*100,
                result[key].high.fmeasure*100
            )
            print(score_text)
        
        return {key: result[key].mid.fmeasure*100 for key in self.score_keys}

In [ ]:
predictions = []
rouge_score = RougeScore()
for i, (input_ids, input_mask, y) in enumerate(test_ds):
    summaries = model.generate(input_ids=input_ids, attention_mask=input_mask)

    pred = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summaries]
    real = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in y]
    
    for pred_sent, real_sent in zip(pred, real):
        rouge_score(pred_sent, real_sent)
        predictions.append(str("pred sentence: " + pred_sent + "\n\n real sentence: " + real_sent))
        
    if i > 10:
        # otherwise it will take ages
        break


rouge_score.result()

rouge1 = 20.53, 95% confidence [19.20, 21.92]
rouge2 = 9.41, 95% confidence [8.38, 10.56]
rougeLsum = 19.59, 95% confidence [18.26, 20.96]


{'rouge1': 20.525347592083808,
 'rouge2': 9.406864066894444,
 'rougeLsum': 19.588098358636906}

In [ ]:
for pred in predictions[:10]:
    print("------")
    print(pred)
    print("------")